In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
%matplotlib inline

In [ ]:
np.random.seed(2355478)
tmp = np.random.poisson(lam=57.3, size=365*10*4)
tmp[np.random.choice(np.arange(tmp.shape[0]),size=365*32, replace=False)] = 0
df = pd.DataFrame(tmp.reshape((40,365)),
                  index=pd.MultiIndex.from_product([np.arange(1,11),np.arange(4)], names=['Customer','Product']),
                  columns=pd.date_range(start='2017-01-01', end='2017-12-31'))

# Assignment 6.1
The DataFrame `df` holds the customers spend on a daily basis for a number of products. The column names are the individual dates in 2017, while the index values are the combinations of customer IDs and product IDs.

Find the average spend per customer between 2017-04-02 and 2017-08-17, both dates included.

In [ ]:
df.loc[:,'2017-04-02':'2017-08-17'].groupby('Customer').sum().sum(axis=1)

# Assignment 6.2
Using the DataFrame `df` again, find the total monthly spend per product. Return a DataFrame where the index is the product ID and the columns are the month number

In [ ]:
tmp = df.reset_index().melt(id_vars=df.index.names)
tmp['month'] = tmp['variable'].dt.month
monthly_spend_per_prod = tmp.groupby(['Product','month'])['value'].sum()
monthly_spend_per_prod.reset_index().pivot(index='Product', columns='month', values='value')

# Assignment 6.3
Create a DataFrame showing the Customer IDs as index and the month number as columns. 

In [ ]:
cust_prod_spend_per_month = (tmp
                             .groupby(['Customer','Product','month'])
                             .sum()
                             .reset_index()
                             .merge(monthly_spend_per_prod
                                    .rename('Total spend')
                                    .to_frame(),
                                    left_on = ['Product','month'],
                                    right_index=True))
cust_prod_spend_per_month['share'] = cust_prod_spend_per_month['value']/cust_prod_spend_per_month['Total spend']

cust_prod_spend_per_month = (cust_prod_spend_per_month
                             .merge(cust_prod_spend_per_month
                                    .groupby(['Customer','month'])['share']
                                    .max()
                                    .rename('max_share')
                                    .to_frame(),
                                    left_on=['Customer','month'],
                                    right_index=True))

(cust_prod_spend_per_month[cust_prod_spend_per_month['share']==cust_prod_spend_per_month['max_share']]
 .pivot(index='Customer',
        columns='month',
        values='Product'))